In [1]:
import re
import os
import pandas as pd
from datetime import datetime, timedelta

if __name__ == "__main__":
    # Get the user's document folder on Windows
    documents_folder = os.path.join(os.path.expanduser("~"), "Documents")

    # Define the main folder for the script within the documents folder
    script_folder = os.path.join(documents_folder, "DATA REVIEW MASTER")

    # Define input and output folders within the main script folder
    input_folder = os.path.join(script_folder, "SOURCE")
    output_folder = os.path.join(script_folder, "OUTPUT")

    # Create the script folder, input folder, and output folder if they don't exist
    for folder in [script_folder, input_folder, output_folder]:
        if not os.path.exists(folder):
            os.makedirs(folder)

    # Function to extract the prefix from a filename
    def extract_prefix(filename):
        return filename.split('_')[0]

    # Function to process a single Excel file
    def process_excel_file(filepath):
        # Load the Excel file
        excel_data = pd.read_excel(filepath, sheet_name=None, dtype=str)
        reviewed_data = {}

        # Process each worksheet
        for sheet_name, df in excel_data.items():
            # Rename worksheets based on conditions
            if len(df.columns) == 46:
                sheet_name = 'individual borrower'
            elif len(df.columns) == 29:
                sheet_name = 'credit information'
            elif len(df.columns) == 20:
                sheet_name = 'corporate borrower'
            elif len(df.columns) == 35:
                sheet_name = 'principal officer'
            elif len(df.columns) == 22:
                sheet_name = 'guarantor information'

            reviewed_data[sheet_name] = df

        return reviewed_data

    # Function to validate BVN format
    def validate_bvn(bvn):
        # Validate BVN format: 11 digits, starts with "22", and not equal to '22222222222'
        return bool(re.match(r'^22\d{9}$', str(bvn))) and str(bvn) != '22222222222'

    # Function to validate RC No format
    def validate_rc_no(rc_no):
        # Rule 1: Values with only letters are not allowed
        rule_1 = not bool(re.match(r'^[^\d]+$', str(rc_no)))

        # Rule 2: Remove values with a single repeated digit
        rule_2 = not any(str(i) * len(str(rc_no)) == str(rc_no) for i in range(10))

        # Rule 3: Remove values with more than eight digits
        rule_3 = len(re.findall(r'\d', str(rc_no))) <= 8

        # Return True if all rules are satisfied, meaning the value is valid
        return rule_1 and rule_2 and rule_3

    # Function to append data count to the Record_Count.xlsx
    def append_data_count(prefix, counts):
        record_count_filepath = os.path.join(output_folder, '01. Record_Count.xlsx')

        if os.path.exists(record_count_filepath):
            existing_counts = pd.read_excel(record_count_filepath)
            counts_df = pd.DataFrame(counts, index=[0])
            updated_counts = pd.concat([existing_counts, counts_df], ignore_index=True)
        else:
            updated_counts = pd.DataFrame(counts, index=[0])

        updated_counts.to_excel(record_count_filepath, index=False)

    # Function to create the Record_Count.xlsx
    def create_record_count(reviewed_data, prefix):
        individual_df = reviewed_data.get('individual borrower', pd.DataFrame())

        # Validate BVN format and remove invalid rows
        individual_df = individual_df[individual_df['BVN No'].apply(validate_bvn)]

        corporate_df = reviewed_data.get('corporate borrower', pd.DataFrame())

        # Validate RC No format and remove invalid rows
        corporate_df = corporate_df[corporate_df['Business Identification No'].apply(validate_rc_no)]

        data_counts = {
            'iocname': prefix,
            'BVN': individual_df['BVN No'].nunique(),
            'RC_NO': corporate_df['Business Identification No'].nunique(),
            'RAW_FAC': reviewed_data.get('credit information', pd.DataFrame()).shape[0],
            'RAW_CONS_SUB': individual_df.shape[0],
            'RAW_CORP_SUB': corporate_df.shape[0]
        }
        append_data_count(prefix, data_counts)
        print(data_counts)

    # Function to create the Extract1 workbook
    def create_extract1(reviewed_data, prefix):
        extract1_sheets = ['credit information', 'individual borrower', 'guarantor information']

        # Get the suffix in 'YYYYMM' format
        last_month = datetime.now() - timedelta(days=30)
        suffix = last_month.strftime('%Y%m')

        extract1_filepath = os.path.join(output_folder, f"{prefix}_cdt_allndvdl_mfi_{suffix}.xlsx")
        with pd.ExcelWriter(extract1_filepath) as writer:
            for sheet_name in extract1_sheets:
                if sheet_name in reviewed_data:
                    df = reviewed_data[sheet_name]
                    if sheet_name == 'credit information':
                        df = df.sort_values(by='Customer ID', ascending=True)
                    elif sheet_name == 'individual borrower':
                        # Validate BVN format and remove invalid rows
                        # df = df[df['BVN No'].apply(validate_bvn)]
                        df = df.sort_values(by='CustomerID', ascending=True)
                    df.to_excel(writer, sheet_name=sheet_name, index=False)

            if 'guarantor information' not in reviewed_data:
                empty_df = pd.DataFrame()
                empty_df.to_excel(writer, sheet_name='guarantor information', index=False)

    # Function to create the Extract2 workbook
    def create_extract2(reviewed_data, prefix):
        extract2_sheets = ['credit information', 'corporate borrower', 'guarantor information', 'principal officer']

        # Get the suffix in 'YYYYMM' format
        last_month = datetime.now() - timedelta(days=30)
        suffix = last_month.strftime('%Y%m')

        extract2_filepath = os.path.join(output_folder, f"{prefix}_cdt_allcorp_mfi_{suffix}.xlsx")
        with pd.ExcelWriter(extract2_filepath) as writer:
            for sheet_name in extract2_sheets:
                if sheet_name in reviewed_data:
                    df = reviewed_data[sheet_name]
                    if sheet_name == 'credit information':
                        df = df.sort_values(by='Customer ID', ascending=True)
                    elif sheet_name == 'corporate borrower':
                        # Apply validate_rc_no to filter out invalid rows
                        #
                        # Apply validate_rc_no to filter out invalid rows
                        #df = df[df['Business Identification No'].apply(validate_rc_no)]
                        df = df.sort_values(by='Business Name', ascending=True)
                    df.to_excel(writer, sheet_name=sheet_name, index=False)

            if 'principal officer' not in reviewed_data:
                empty_df = pd.DataFrame()
                empty_df.to_excel(writer, sheet_name='principal officer', index=False)

            if 'guarantor information' not in reviewed_data:
                empty_df = pd.DataFrame()
                empty_df.to_excel(writer, sheet_name='guarantor information', index=False)

    # Process each file in the source directory
    for filename in os.listdir(input_folder):
        if filename.endswith((".xlsx", ".xls", ".xlb")):
            file_path = os.path.join(input_folder, filename)
            prefix = (extract_prefix(filename).lower())
            reviewed_data = process_excel_file(file_path)

            # Data transformation for 'credit information' dataframe
            if 'credit information' in reviewed_data:
                try:
                    credit_info_df = reviewed_data['credit information']
                    credit_info_df['Previous Account number'] = ''
                    credit_info_df['Previous Name'] = ''
                    credit_info_df['Previous Customer ID'] = ''
                    credit_info_df['Previous Branch code'] = ''
                    credit_info_df['Account Status'] = credit_info_df['Account Status'].fillna('OPEN')
                    credit_info_df['Currency'] = credit_info_df['Currency'].fillna('NAIRA')
                    credit_info_df['Loan (Facility) type'] = credit_info_df['Loan (Facility) type'].fillna('OTHERS')
                    last_month = datetime.now() - timedelta(days=30)
                    #credit_info_df['Last payment date'] = credit_info_df['Last payment date'].apply(lambda x: x if (pd.to_datetime(x) <= last_month and x) else '')
                    credit_info_df['Last payment date'] = ''
                    credit_info_df['Date of loan (facility) disbursement/Loan effective date'] = ''
                    credit_info_df['Loan Security Status'] = credit_info_df['Loan Security Status'].apply(lambda x: 'SECURED' if x.lower() in ['yes', 'secure'] else 'UNSECURED')
                    reviewed_data['credit information'] = credit_info_df
                except Exception as e:
                    print(f"Error processing 'individual borrower' dataframe for file: {filename}")
                    print(f"Error details: {str(e)}")

            # Data transformation for 'individual borrower' dataframe
            if 'individual borrower' in reviewed_data:
                try:
                    individual_borrower_df = reviewed_data['individual borrower']
                    individual_borrower_df['Branch Code'] = individual_borrower_df['Branch Code'].fillna('001')
                    individual_borrower_df['Nationality'] = individual_borrower_df['Nationality'].apply(lambda x: 'NGR' if pd.isna(x) or x != 'NG' else x)
                    individual_borrower_df['Primary Country'] = individual_borrower_df['Primary Country'].apply(lambda x: 'NGR' if pd.isna(x) or x != 'NG' else x)
                    # Validate BVN format and remove invalid rows
                    #individual_borrower_df = individual_borrower_df[individual_borrower_df['BVN No'].apply(validate_bvn)]
                    individual_borrower_df['Primary Address Line 1'] = individual_borrower_df.apply(lambda row: row['Secondary Address Line 1'] if pd.isna(row['Primary Address Line 1']) else row['Primary Address Line 1'], axis=1)
                    reviewed_data['individual borrower'] = individual_borrower_df
                except Exception as e:
                    print(f"Error processing 'individual borrower' dataframe for file: {filename}")
                    print(f"Error details: {str(e)}")

            # Data transformation for 'corporate borrower' dataframe
            if 'corporate borrower' in reviewed_data:
                try:
                    corporate_borrower_df = reviewed_data['corporate borrower']
                    corporate_borrower_df["Customer's Branch Code"] = corporate_borrower_df["Customer's Branch Code"].fillna('001')
                    # Remove rows with missing or invalid 'Business Identification No'
                    #corporate_borrower_df = corporate_borrower_df.dropna(subset=['Business Identification No'])
                    corporate_borrower_df['Business Office Address Line 1'] = corporate_borrower_df.apply(lambda row: row['Secondary Address Line 1'] if pd.isna(row['Business Office Address Line 1']) else row['Business Office Address Line 1'], axis=1)
                    reviewed_data['corporate borrower'] = corporate_borrower_df
                except Exception as e:
                    print(f"Error processing file: {filename}")
                    print(f"Error details: {str(e)}")

            create_record_count(reviewed_data, prefix)
            create_extract1(reviewed_data, prefix)
            create_extract2(reviewed_data, prefix)

    print("Processing completed.")


{'iocname': 'agosasa', 'BVN': 1065, 'RC_NO': 64, 'RAW_FAC': 7282, 'RAW_CONS_SUB': 1097, 'RAW_CORP_SUB': 86}
Processing completed.
